# Part4: 训练模型
---

##### 输入：一个 csv 文件 -- 训练验证数据集   
##### 输出：一个 onnx 模型文件
---

In [1]:
from train_model import *
from utils import labels
import warnings
warnings.filterwarnings("ignore")

C:\Users\1\miniconda3\envs\_tf24\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 模型训练配置参数

In [2]:
# 全量数据训练轮数，默认为  3
EPOCHS = 2
# 学习率，默认值
LR = 2e-5
# 每次加载到内存中进行训练的数据量，可根据显卡大小调整，越大效果越好
batch_size = 148
# 模型保存地址
save_path = './models/model_test.pth'
# 训练数据路径
path_train = "./dataset/数据增强/东亚全量告警-已增强-修改后.csv"
# onnx格式保存路径
onnx_path = "./models/dy_model_new.onnx"
# 类别分布配置文件路径
config_path = "./config/标签类别映射/config72.json"
# 分类类别数
num_classes = 72
# 随机失活比率
dropout = 0.5

## 自动适配GPU或CPU

In [3]:
use_mps = torch.backends.mps.is_available()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "mps" if use_mps else "cpu")

## 加载数据

In [5]:
df_train = pd.read_csv(path_train)

# 注意：用于验证，上生产注释该语句
df_train = df_train.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=1, random_state=0))

df_train = df_train[:56000]
print(df_train.shape)
df_train, df_val, df_test = np.split(df_train.sample(frac=1, random_state=42),[int(.8 * len(df_train)), int(.9 * len(df_train))])
print(len(df_train), len(df_val), len(df_test))
LABELS = labels(config_path)

(56000, 2)
44800 5600 5600


## 训练模型

In [6]:
model = BertClassifier(dropout=dropout, out_dims=num_classes)
model = model.to(device)
train(model, df_train, df_val, LR, EPOCHS, batch_size, save_path, num_classes, LABELS, onnx_path)
evaluate(model, df_test, batch_size, LABELS) 

Some weights of the model checkpoint at ./pretrain_models/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                          | 0/303 [00:00<?, ?it/s]2024-01-11 15:17:4

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Test Accuracy:  0.998


In [ ]:
# 所选客户:    辽宁农信告警数据
# 数据量:      48337
# 标签类别:    40类
# GPU:        3070Ti
# epochs:     2
# start time: 16:55:24
# end time:   17:44:05
# Used Time:  49分钟 (即: 24.5 min/epoch)
# Accuracy:   0.99